<a href="https://colab.research.google.com/github/S1m0nCi/Durhack-2022/blob/main/Durhack_QRT_Stock_Prediction_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_decision_forests
!pip install wurlitzer

%tensorflow_version 2.x

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [ ]:
from tensorflow import keras

In [ ]:
input_train_df = pd.read_csv('https://durhack.qrt.services/data/x_train.csv')
output_train_df = pd.read_csv('https://durhack.qrt.services/data/y_train.csv')

input_test_df = pd.read_csv('https://durhack.qrt.services/data/x_test.csv')

NUMERIC_COLUMNS = ['DATE', 'STOCK', 'INDUSTRY', 'INDUSTRY_GROUP', 'SUB_INDUSTRY', 'SECTOR']
CATEGORICAL_COLUMNS = ['RET']
# We do not need to convert from categorical to numeric columns because all columns are already numeric. 

for i in range(1,21):
  NUMERIC_COLUMNS.append('RET_' + str(i))
for i in range(1,21):
  NUMERIC_COLUMNS.append('VOLUME_' + str(i))


input_train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(input_train_df) # Not quite correct as we need a boolean label.
output_train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(output_train_df)
input_test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(input_test_df)

# 1. Join the two dataframes to make the solution column be in the actual column OR create a solution column of TRUE and FALSE using RET_1, RET_2, ... and so on.
# 2. Then the model is easy to train and compile - This cell will be for data analysis and the next will be for creating the model. 


In [ ]:
train_df = input_train_df 

# The below is not what we were supposed to do - perhaps.
extracted_col = output_train_df['RET']


train_df = train_df.join(extracted_col)

train_df.head()

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label = "RET")

In [ ]:
baseline_df = train_df[['DATE', 'STOCK', 'INDUSTRY', 'INDUSTRY_GROUP', 'SUB_INDUSTRY', 'SECTOR']].copy()

extracted_col = input_train_df['RET_1']
extracted_col_1 = np.sign(extracted_col)
extracted_col_1[extracted_col_1 < 0] = 0 

extracted_col_2 = input_train_df['VOLUME_1']
training_part_1 = baseline_df.join(extracted_col_1)
training_part = training_part_1.join(extracted_col_2)  
training_part = training_part.apply (pd.to_numeric, errors='coerce')
training_part = training_part.dropna()

train_part_ds = tfdf.keras.pd_dataframe_to_tf_dataset(training_part, label = "RET_1")



In [ ]:
# We need to work with the dataframe
# We create a baseline list
# Chnge positive RET to TRUE or even to 1
# We retrain the model for every ret-volume pair
# Use join
baseline_df = train_df[['DATE', 'STOCK', 'INDUSTRY', 'INDUSTRY_GROUP', 'SUB_INDUSTRY', 'SECTOR']].copy()
ds_list = []
# Creating a collected list is more efficient
collected_df = pd.DataFrame()

for num in range(1,6):
  extracted_col = input_train_df['RET_'+str(num)]
  extracted_col_1 = np.sign(extracted_col)
  extracted_col_1[extracted_col_1 < 0] = 0 
  extracted_col_df = pd.DataFrame(extracted_col_1)
  extracted_col_df.rename(columns = {'RET_'+str(num): 'RET'}, inplace = True)
  training_part_1 = baseline_df.join(extracted_col_df)

  extracted_col_2 = input_train_df['VOLUME_'+str(num)]
  extracted_col_2_df = pd.DataFrame(extracted_col_2)
  extracted_col_2_df.rename(columns = {'VOLUME_'+str(num): 'VOLUME'}, inplace = True)

  training_part = training_part_1.join(extracted_col_2_df)  
  training_part = training_part.apply(pd.to_numeric, errors='coerce')
  training_part = training_part.dropna()

  collected_df = collected_df.append(training_part, ignore_index = True)

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(collected_df, label = "RET")



In [ ]:
model = tfdf.keras.RandomForestModel(
    num_trees = 500,
    max_depth = 8    
)  # Configure this model's details such as the number of trees.


model.fit(train_ds)

model.compile(metrics=["accuracy"])

https://www.tensorflow.org/decision_forests/tutorials/predict_colab 

In [ ]:
model.predict(input_test_ds)

In [ ]:
# Format the test data - the code may crash
collected_df_test = pd.DataFrame()

for num in range(1,21):
  extracted_col = input_test_df['RET_'+str(num)]
  extracted_col_1 = np.sign(extracted_col)
  extracted_col_1[extracted_col_1 < 0] = 0 
  extracted_col_df = pd.DataFrame(extracted_col_1)
  extracted_col_df.rename(columns = {'RET_'+str(num): 'RET'}, inplace = True)
  testing_part_1 = baseline_df.join(extracted_col_df)

  extracted_col_2 = input_test_df['VOLUME_'+str(num)]
  extracted_col_2_df = pd.DataFrame(extracted_col_2)
  extracted_col_2_df.rename(columns = {'VOLUME_'+str(num): 'VOLUME'}, inplace = True)

  testing_part = testing_part_1.join(extracted_col_2_df)  
  testing_part = testing_part.apply(pd.to_numeric, errors='coerce')
  testing_part = testing_part.dropna()

  collected_df_test = collected_df.append(testing_part, ignore_index = True)

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(collected_df_test, label = "RET")

In [ ]:
import csv

#Change TRUE to 1 and FALSE to 0.
model.evaluate(test_ds)

fil = open('test_rand.csv', 'w')


"""
for prediction_number in model.predict(input_test_ds):
  if prediction_number """

